# Inside `ulinalg`
Inside `ulinalg`, you would define a method consisting of an argument replacer and argument extractor.

- The argument extractor is the simpler of the two: It "extracts" the array arguments from the method as a `tuple`.
- The argument replacer "replaces" the array arguments inside args, kwargs with the supplied arrays.

In [1]:
import uarray as ua
from uarray import all_of_type
import unumpy as unp

In [2]:
def solve_argreplacer(args, kwargs, arrays):
    out_args = arrays + args[2:]
    return out_args, kwargs

@unp.create_numpy(solve_argreplacer)
@all_of_type(unp.ndarray)
def solve(a, b, sym_pos=False, lower=False, overwrite_a=False, overwrite_b=False, debug=None, check_finite=True, assume_a='gen', transposed=False):
    return (a, b)

# Inside a Numpy backend for `ulinalg` (ideally, `scipy.linalg` itself)
Here, you register the implementation for the backend itself.

In [3]:
import unumpy.numpy_backend as NumpyBackend
import scipy.linalg as linalg
ua.set_global_backend(NumpyBackend)

In [4]:
NumpyBackend._implementations[solve] = linalg.solve

# Inside PyTorch or a PyTorch backend for `ulinalg` (ideally, PyTorch itself)
Here, you have to perform some translation, because the PyTorch API isn't 1:1 with the NumPy API. See [this documentation page](https://pytorch.org/docs/stable/torch.html#torch.solve). In practice, you would select the function based on the input arguments, but let's ignore that for now.

In [5]:
import unumpy.torch_backend as TorchBackend
import torch

ua.register_backend(TorchBackend)

In [6]:
def solve_impl(a, b, sym_pos=False, lower=False, overwrite_a=False, overwrite_b=False, debug=None, check_finite=True, assume_a='gen', transposed=False):
    return torch.solve(b, a)[0]

In [7]:
TorchBackend._implementations[solve] = solve_impl

# User Experience
The user simply imports the right library and uses everything as normal. Let's try solving this fruit puzzle:

<img src="fruit-puzzle.jpg" width=400>

In [8]:
a = [[3.0, 0.0, 0.0], [1.0, 2.0, 0.0], [0.0, 1.0, -2.0]]
b = [[30.0], [18.0], [2.0]]

In [9]:
ta = torch.tensor(a, dtype=torch.float32); tb = torch.tensor(b, dtype=torch.float32)

tx = solve(ta, tb)
tsol = tx[0] + tx[1] + tx[2]

In [10]:
print(tsol)
print(type(tsol))

[15.]
<class 'numpy.ndarray'>


In [11]:
import numpy as np

na = np.array(a); nb = np.array(b)

nx = solve(na, nb)
nsol = nx[0] + nx[1] + nx[2]

In [12]:
print(nsol)
print(type(nsol))

[15.]
<class 'numpy.ndarray'>


## Manually choosing the back-end
It's also possible to use context managers to safely set the back-end.

In [13]:
import uarray as ua
with ua.set_backend(NumpyBackend, coerce=True):
    print(type(solve(a, b)))
        
with ua.set_backend(TorchBackend, coerce=True):
    print(type(solve(ta, tb)))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>
